# Flink Data Trace + Integration Test Notebook

This notebook validates pipeline correctness by querying data in strict order:

1. Raw ingress (`streaming_events`, DLQ, quarantine)
2. Typed tables (`ai_stream_status`, `stream_trace_events`, `ai_stream_events`, `stream_ingest_metrics`)
3. Silver projections (`fact_stream_*`)
4. Stateful lifecycle facts (`fact_workflow_*`)
5. Rollups (`agg_*`)
6. API views (`v_api_*`)

It also runs SQL assertion suites and scenario-candidate checks used by the CLI harness.

In [1]:
# If needed once:
# %pip install clickhouse-connect pandas


In [2]:
from __future__ import annotations

import os
from datetime import datetime, timedelta, timezone
from pathlib import Path

import clickhouse_connect
import pandas as pd


def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'tests' / 'integration' / 'sql').is_dir():
            return candidate
    raise FileNotFoundError('Could not locate repo root containing tests/integration/sql')


CH_HOST = os.getenv('CH_HOST', 'clickhouse.livepeer.cloud')
CH_PORT = int(os.getenv('CH_PORT', '8123'))
CH_DATABASE = os.getenv('CH_DATABASE', 'livepeer_analytics')
CH_USER = os.getenv('CH_USER', 'analytics_user')
CH_PASSWORD = os.getenv('CH_PASSWORD', 'analytics_password')
CH_SECURE = os.getenv('CH_SECURE', '').lower() in {'1', 'true', 'yes'}

VALIDATION_LOOKBACK_HOURS = int(os.getenv('VALIDATION_LOOKBACK_HOURS', '24'))
LIMIT_PER_SCENARIO = int(os.getenv('LIMIT_PER_SCENARIO', '5'))

REPO_ROOT = find_repo_root(Path.cwd().resolve())
SQL_DIR = REPO_ROOT / 'tests' / 'integration' / 'sql'

TO_TS = datetime.now(timezone.utc)
FROM_TS = TO_TS - timedelta(hours=VALIDATION_LOOKBACK_HOURS)

print(f'Repo root: {REPO_ROOT}')
print(f'SQL dir: {SQL_DIR}')
print(f'Window UTC ({VALIDATION_LOOKBACK_HOURS}h): {FROM_TS.isoformat()} -> {TO_TS.isoformat()}')
import ipywidgets as widgets
from IPython.display import display


Repo root: ${REPO_ROOT}
SQL dir: ${REPO_ROOT}/tests/integration/sql
Window UTC (24h): 2026-02-22T18:32:12.473407+00:00 -> 2026-02-23T18:32:12.473407+00:00


In [3]:
client = clickhouse_connect.get_client(
    host=CH_HOST,
    port=CH_PORT,
    username=CH_USER,
    password=CH_PASSWORD,
    database=CH_DATABASE,
    secure=CH_SECURE,
)

QUERY_SETTINGS = {
    'max_execution_time': int(os.getenv('CH_MAX_EXECUTION_TIME_SEC', '30')),
    'max_threads': int(os.getenv('CH_MAX_THREADS', '4')),
    'max_rows_to_read': int(os.getenv('CH_MAX_ROWS_TO_READ', '5000000')),
    'read_overflow_mode': 'break',
}

params = {
    'from_ts': FROM_TS.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
    'to_ts': TO_TS.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
    'limit_per_scenario': LIMIT_PER_SCENARIO,
}


def query_df(sql: str, parameters: dict | None = None) -> pd.DataFrame:
    result = client.query(sql, parameters=parameters or {}, settings=QUERY_SETTINGS)
    return pd.DataFrame(result.result_rows, columns=result.column_names)


def parse_blocks(path: str | Path, marker: str) -> list[tuple[str, str]]:
    blocks = []
    current_name = None
    current_lines = []
    for raw_line in Path(path).read_text(encoding='utf-8').splitlines():
        if raw_line.startswith(marker):
            if current_name and current_lines:
                sql = '\n'.join(current_lines).strip().rstrip(';')
                if sql:
                    blocks.append((current_name, sql))
            current_name = raw_line.split(':', 1)[1].strip()
            current_lines = []
            continue
        if current_name is not None:
            current_lines.append(raw_line)
    if current_name and current_lines:
        sql = '\n'.join(current_lines).strip().rstrip(';')
        if sql:
            blocks.append((current_name, sql))
    return blocks


## Ordered Pipeline Trace (Raw -> API)

In [4]:
flow_file = SQL_DIR / 'trace_pipeline_flow.sql'
flow_blocks = parse_blocks(flow_file, '-- QUERY:')
print(f'Loaded {len(flow_blocks)} ordered queries from {flow_file}')

flow_results: dict[str, pd.DataFrame] = {}
for name, sql in flow_blocks:
    df = query_df(sql, params)
    flow_results[name] = df
    print(f'\n=== {name} ===')
    display(df.head(20))


Loaded 9 ordered queries from ${REPO_ROOT}/tests/integration/sql/trace_pipeline_flow.sql

=== 01_raw_ingest ===


,object_name,rows_window,min_ts,max_ts
0,streaming_events_quarantine,0,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000
1,streaming_events_dlq,627,2026-02-22 18:32:52.143,2026-02-23 18:17:52.140
2,streaming_events,24337,2026-02-22 18:32:23.664,2026-02-23 18:31:42.353



=== 02_typed_tables ===


,object_name,rows_window,min_ts,max_ts
0,ai_stream_events,4918,2026-02-22 18:40:27.408,2026-02-23 18:27:17.653
1,ai_stream_status,2624,2026-02-22 18:41:02.835,2026-02-23 18:25:46.227
2,network_capabilities,19697,2026-02-22 18:32:32.561,2026-02-23 18:24:30.632
3,stream_ingest_metrics,0,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000
4,stream_trace_events,10322,2026-02-22 18:40:27.406,2026-02-23 18:27:17.653



=== 03_silver_projection_counts ===


...

,object_name,rows_window
0,v_api_gpu_metrics,487
1,v_api_network_demand,157
2,v_api_sla_compliance,477



=== 08_gpu_view_parity ===


,joined_rows,mean_abs_diff_fps,max_abs_diff_fps
0,0,NaN,0.0



=== 09_sla_view_parity ===


,joined_rows,total_known_diff,total_unexcused_diff,total_swapped_diff
0,477,0,0,0


## Integration Assertions (CI-aligned)

In [5]:
def run_assertion_file(path: str | Path) -> pd.DataFrame:
    tests = parse_blocks(path, '-- TEST:')
    rows = []
    for name, sql in tests:
        try:
            df = query_df(sql, params)
        except Exception as exc:
            rows.append({'test_name': name, 'failed_rows': 1, 'status': 'FAIL', 'error': str(exc)})
            continue

        if df.empty:
            rows.append({'test_name': name, 'failed_rows': 1, 'status': 'FAIL', 'error': 'No rows'})
            continue

        row = df.iloc[0].to_dict()
        failed = int(row.get('failed_rows', 1))
        row['test_name'] = name
        row['status'] = 'PASS' if failed == 0 else 'FAIL'
        rows.append(row)

    out = pd.DataFrame(rows)
    cols = ['test_name', 'status', 'failed_rows'] + [c for c in out.columns if c not in {'test_name', 'status', 'failed_rows'}]
    return out[cols]

pipeline_assertions = run_assertion_file(SQL_DIR / 'assertions_pipeline.sql')
display(pipeline_assertions)

pipeline_failures = pipeline_assertions[pipeline_assertions['status'] == 'FAIL']
print(f'Pipeline assertion failures: {len(pipeline_failures)}')


,test_name,status,failed_rows,missing_objects,missing_mvs,caps_rows,dim_caps_rows,adv_rows,dim_adv_rows,mc_rows,...,total_rows,swapped_without_evidence,orphan_param_updates,joined_rows,mean_abs_diff_fps,max_abs_diff_fps,total_known_diff,total_unexcused_diff,total_swapped_diff,rows_checked
0,raw_events_present,PASS,0.0,[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,capability_dimension_mvs_present,PASS,0.0,NaN,[],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,capability_dimensions_projecting,PASS,0.0,NaN,NaN,19697.0,16188.0,0.0,0.0,8255.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,session_fact_present,PASS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,status_raw_to_silver_projection,PASS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,trace_raw_to_silver_projection,PASS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ingest_raw_to_silver_projection,PASS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,session_final_uniqueness,PASS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,workflow_session_has_identifier,PASS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,swapped_sessions_have_evidence,PASS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Pipeline assertion failures: 0


In [6]:
api_readiness_assertions = run_assertion_file(SQL_DIR / 'assertions_api_readiness.sql')
display(api_readiness_assertions)

api_readiness_failures = api_readiness_assertions[api_readiness_assertions['status'] == 'FAIL']
print(f'API readiness assertion failures: {len(api_readiness_failures)}')


,test_name,status,failed_rows,missing_views,rows_checked,missing_columns,low_sample_windows,joined_rows
0,api_views_present,PASS,0,[],NaN,NaN,NaN,NaN
1,gpu_metrics_keys_not_null,PASS,0,NaN,487.0,NaN,NaN,NaN
2,gpu_metrics_rollup_fields_consistent,PASS,0,NaN,487.0,NaN,NaN,NaN
3,gpu_metrics_latency_fields_nonnegative,PASS,0,NaN,487.0,NaN,NaN,NaN
4,gpu_metrics_startup_seconds_matches_ms,PASS,0,NaN,487.0,NaN,NaN,NaN
5,network_demand_hourly_grain,PASS,0,NaN,157.0,NaN,NaN,NaN
6,network_demand_by_gpu_hourly_grain,PASS,0,NaN,531.0,NaN,NaN,NaN
7,network_demand_by_gpu_required_columns_present,PASS,0,NaN,NaN,[],NaN,NaN
8,network_demand_by_gpu_capacity_fields_nonnegative,PASS,0,NaN,531.0,NaN,NaN,NaN
9,network_demand_additive_fields_nonnegative,PASS,0,NaN,157.0,NaN,NaN,NaN


API readiness assertion failures: 0


## API Readiness Failure Drill-Down
This section shows offending rows for failed API readiness checks so failures are explainable (contract bug vs sparse telemetry).


In [7]:
# Drill-down queries for common readiness failures
readiness_checks = {
    'gpu_metrics_keys_not_null': '''
        SELECT *
        FROM livepeer_analytics.v_api_gpu_metrics
        WHERE window_start >= {from_ts:DateTime64(3)}
          AND window_start < {to_ts:DateTime64(3)}
          AND (
            orchestrator_address = ''
            OR pipeline = ''
            OR ifNull(gpu_id, '') = ''
            OR window_start IS NULL
          )
        ORDER BY window_start DESC
        LIMIT 50
    ''',
    'gpu_metrics_rollup_fields_consistent': '''
        SELECT
          window_start,
          orchestrator_address,
          pipeline,
          model_id,
          gpu_id,
          known_sessions,
          unexcused_sessions,
          swapped_sessions,
          failure_rate,
          ifNull(unexcused_sessions / nullIf(known_sessions, 0), 0) AS recomputed_failure_rate,
          swap_rate,
          ifNull(swapped_sessions / nullIf(known_sessions, 0), 0) AS recomputed_swap_rate
        FROM livepeer_analytics.v_api_gpu_metrics
        WHERE window_start >= {from_ts:DateTime64(3)}
          AND window_start < {to_ts:DateTime64(3)}
          AND (
            abs(failure_rate - ifNull(unexcused_sessions / nullIf(known_sessions, 0), 0)) > 0.000001
            OR abs(swap_rate - ifNull(swapped_sessions / nullIf(known_sessions, 0), 0)) > 0.000001
          )
        ORDER BY window_start DESC
        LIMIT 50
    ''',
    'network_demand_by_gpu_hourly_grain': '''
        SELECT *
        FROM livepeer_analytics.v_api_network_demand_by_gpu
        WHERE window_start >= {from_ts:DateTime64(3)}
          AND window_start < {to_ts:DateTime64(3)}
          AND toMinute(window_start) != 0
        ORDER BY window_start DESC
        LIMIT 50
    ''',
    'network_demand_by_gpu_capacity_fields_nonnegative': '''
        SELECT *
        FROM livepeer_analytics.v_api_network_demand_by_gpu
        WHERE window_start >= {from_ts:DateTime64(3)}
          AND window_start < {to_ts:DateTime64(3)}
          AND (
            inference_minutes_by_gpu_type < 0
            OR used_inference_minutes < 0
            OR available_capacity_minutes < 0
            OR capacity_rate < 0
            OR capacity_rate > 1.5
          )
        ORDER BY window_start DESC
        LIMIT 50
    ''',
    'sla_compliance_rollup_safe': '''
        SELECT
          window_start,
          orchestrator_address,
          pipeline,
          model_id,
          gpu_id,
          known_sessions,
          unexcused_sessions,
          swapped_sessions,
          success_ratio,
          ifNull(1 - (unexcused_sessions / nullIf(known_sessions, 0)), 0) AS recomputed_success_ratio,
          no_swap_ratio,
          ifNull(1 - (swapped_sessions / nullIf(known_sessions, 0)), 0) AS recomputed_no_swap_ratio
        FROM livepeer_analytics.v_api_sla_compliance
        WHERE window_start >= {from_ts:DateTime64(3)}
          AND window_start < {to_ts:DateTime64(3)}
          AND (
            abs(success_ratio - ifNull(1 - (unexcused_sessions / nullIf(known_sessions, 0)), 0)) > 0.000001
            OR abs(no_swap_ratio - ifNull(1 - (swapped_sessions / nullIf(known_sessions, 0)), 0)) > 0.000001
          )
        ORDER BY window_start DESC
        LIMIT 50
    '''
}

if 'api_readiness_assertions' in globals():
    failed_tests = api_readiness_assertions.loc[api_readiness_assertions['status'] == 'FAIL', 'test_name'].tolist()
    print('Failed readiness tests:', failed_tests)
    for t in failed_tests:
        sql = readiness_checks.get(t)
        if not sql:
            print(f'\n{t}: no drill-down query registered')
            continue
        print(f'\n=== {t} offending rows ===')
        df = query_df(sql, params)
        display(df)
else:
    print('Run API readiness assertions first.')


Failed readiness tests: []


## Serving Schema Sanity
Quick schema contract check for serving views to catch column naming drift before API integration.


In [8]:
required_cols = {
    'v_api_gpu_metrics': ['window_start', 'orchestrator_address', 'pipeline', 'model_id', 'gpu_id', 'status_samples', 'known_sessions'],
    'v_api_network_demand': ['window_start', 'gateway', 'pipeline', 'total_streams', 'total_sessions', 'total_inference_minutes'],
    'v_api_network_demand_by_gpu': ['window_start', 'gateway', 'orchestrator_address', 'pipeline', 'model_id', 'gpu_id', 'capacity_rate'],
    'v_api_sla_compliance': ['window_start', 'orchestrator_address', 'pipeline', 'model_id', 'gpu_id', 'sla_score'],
}

rows = []
for view_name, must_have in required_cols.items():
    d = query_df(f"DESCRIBE TABLE livepeer_analytics.{view_name}")
    cols = set(d['name'].tolist())
    missing = [c for c in must_have if c not in cols]
    rows.append({
        'view_name': view_name,
        'column_count': len(cols),
        'missing_required_columns': missing,
        'status': 'PASS' if not missing else 'FAIL'
    })

schema_check_df = pd.DataFrame(rows)
display(schema_check_df)

for view_name in required_cols:
    print(f'\n=== {view_name} columns ===')
    display(query_df(f"DESCRIBE TABLE livepeer_analytics.{view_name}"))


,view_name,column_count,missing_required_columns,status
0,v_api_gpu_metrics,31,[],PASS
1,v_api_network_demand,17,[],PASS
2,v_api_network_demand_by_gpu,19,[],PASS
3,v_api_sla_compliance,14,[],PASS



=== v_api_gpu_metrics columns ===


,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,window_start,DateTime('UTC'),,,,,
1,orchestrator_address,String,,,,,
2,pipeline,String,,,,,
3,model_id,Nullable(String),,,,,
4,gpu_id,Nullable(String),,,,,
5,region,Nullable(String),,,,,
6,gpu_name,Nullable(String),,,,,
7,gpu_memory_total,Nullable(UInt64),,,,,
8,runner_version,Nullable(String),,,,,
9,cuda_version,Nullable(String),,,,,



=== v_api_network_demand columns ===


,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,window_start,DateTime('UTC'),,,,,
1,gateway,String,,,,,
2,region,Nullable(String),,,,,
3,pipeline,String,,,,,
4,total_streams,UInt64,,,,,
5,total_sessions,UInt64,,,,,
6,total_inference_minutes,Float64,,,,,
7,avg_output_fps,Float64,,,,,
8,known_sessions,UInt64,,,,,
9,served_sessions,UInt64,,,,,



=== v_api_network_demand_by_gpu columns ===


,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,window_start,DateTime('UTC'),,,,,
1,gateway,String,,,,,
2,orchestrator_address,String,,,,,
3,region,Nullable(String),,,,,
4,pipeline,String,,,,,
5,model_id,Nullable(String),,,,,
6,gpu_id,Nullable(String),,,,,
7,gpu_type,String,,,,,
8,total_streams,UInt64,,,,,
9,total_sessions,UInt64,,,,,



=== v_api_sla_compliance columns ===


,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,window_start,DateTime('UTC'),,,,,
1,orchestrator_address,String,,,,,
2,pipeline,String,,,,,
3,model_id,Nullable(String),,,,,
4,gpu_id,Nullable(String),,,,,
5,region,Nullable(String),,,,,
6,known_sessions,UInt64,,,,,
7,success_sessions,UInt64,,,,,
8,excused_sessions,UInt64,,,,,
9,unexcused_sessions,UInt64,,,,,


## Raw -> Silver Correlation Checks

These rows show that raw typed records are carried into silver facts using `source_event_uid` correlation.

In [9]:
projection_checks = pipeline_assertions[pipeline_assertions['test_name'].str.contains('_raw_to_silver_projection', na=False)]
display(projection_checks[['test_name', 'status', 'failed_rows', 'typed_rows', 'projected_rows', 'missing_in_silver']])


,test_name,status,failed_rows,typed_rows,projected_rows,missing_in_silver
4,status_raw_to_silver_projection,PASS,0.0,2624.0,2624.0,0.0
5,trace_raw_to_silver_projection,PASS,0.0,10322.0,10322.0,0.0
6,ingest_raw_to_silver_projection,PASS,0.0,0.0,0.0,0.0


## Contract Coverage Profile (Telemetry Completeness)
Profiles key sparsity signals to separate telemetry gaps from serving-model bugs.


In [10]:
coverage_sql = '''
SELECT *
FROM
(
  SELECT
    'v_api_gpu_metrics' AS object_name,
    count() AS rows_window,
    countIf(pipeline = '') AS empty_pipeline_rows,
    countIf(ifNull(gpu_id, '') = '') AS empty_gpu_rows,
    countIf(status_samples = 0) AS zero_status_sample_rows,
    CAST(NULL AS Nullable(UInt64)) AS empty_gateway_rows
  FROM livepeer_analytics.v_api_gpu_metrics
  WHERE window_start >= {from_ts:DateTime64(3)}
    AND window_start < {to_ts:DateTime64(3)}

  UNION ALL

  SELECT
    'v_api_network_demand' AS object_name,
    count() AS rows_window,
    countIf(pipeline = '') AS empty_pipeline_rows,
    CAST(NULL AS Nullable(UInt64)) AS empty_gpu_rows,
    CAST(NULL AS Nullable(UInt64)) AS zero_status_sample_rows,
    countIf(gateway = '') AS empty_gateway_rows
  FROM livepeer_analytics.v_api_network_demand
  WHERE window_start >= {from_ts:DateTime64(3)}
    AND window_start < {to_ts:DateTime64(3)}
)
ORDER BY object_name
'''

display(query_df(coverage_sql, params))

hourly_sparsity_sql = '''
SELECT
  toStartOfInterval(sample_ts, INTERVAL 1 HOUR) AS window_start,
  count() AS status_rows,
  countIf(output_fps > 0) AS fps_positive_rows,
  countIf(output_fps = 0) AS fps_zero_rows,
  countIf(gateway = '') AS empty_gateway_rows
FROM livepeer_analytics.fact_stream_status_samples
WHERE sample_ts >= {from_ts:DateTime64(3)}
  AND sample_ts < {to_ts:DateTime64(3)}
GROUP BY window_start
ORDER BY window_start DESC
'''

print('\nStatus sample sparsity by hour:')
display(query_df(hourly_sparsity_sql, params))


,object_name,rows_window,empty_pipeline_rows,empty_gpu_rows,zero_status_sample_rows,empty_gateway_rows
0,v_api_gpu_metrics,487,0,0.0,123.0,NaN
1,v_api_network_demand,157,0,NaN,NaN,0.0



Status sample sparsity by hour:


,window_start,status_rows,fps_positive_rows,fps_zero_rows,empty_gateway_rows
0,2026-02-23 18:00:00,15,12,3,0
1,2026-02-23 17:00:00,113,72,41,0
2,2026-02-23 16:00:00,95,59,36,0
3,2026-02-23 15:00:00,87,53,34,0
4,2026-02-23 14:00:00,97,63,34,0
5,2026-02-23 13:00:00,91,53,38,0
6,2026-02-23 12:00:00,137,86,51,0
7,2026-02-23 11:00:00,78,47,31,0
8,2026-02-23 10:00:00,102,61,41,0
9,2026-02-23 09:00:00,137,82,55,0


## Scenario Candidate Discovery

In [11]:
scenario_blocks = parse_blocks(SQL_DIR / 'scenario_candidates.sql', '-- QUERY:')
scenario_candidates: dict[str, pd.DataFrame] = {}
for name, sql in scenario_blocks:
    df = query_df(sql, params)
    scenario_candidates[name] = df
    print(f'\n{name}: {len(df)} candidate rows')
    display(df.head(10))



scenario_1_clean_success_no_swap_fps_gt_12: 5 candidate rows


,scenario_name,workflow_session_id,stream_id,request_id,session_start_ts,session_end_ts,avg_output_fps,segment_orchestrators
0,scenario_1_clean_success_no_swap_fps_gt_12,aiJobTesterStream-1771871101314442233|8381f156,aiJobTesterStream-1771871101314442233,8381f156,2026-02-23 18:25:05.925,2026-02-23 18:25:49.266,13.758626,1
1,scenario_1_clean_success_no_swap_fps_gt_12,aiJobTesterStream-1771871024379358396|10d21483,aiJobTesterStream-1771871024379358396,10d21483,2026-02-23 18:23:48.785,2026-02-23 18:25:03.567,15.970694,1
2,scenario_1_clean_success_no_swap_fps_gt_12,aiJobTesterStream-1771868983885107296|0ba52730,aiJobTesterStream-1771868983885107296,0ba52730,2026-02-23 17:49:49.489,2026-02-23 17:50:34.878,12.591030,1
3,scenario_1_clean_success_no_swap_fps_gt_12,aiJobTesterStream-1771867529384143969|172e98a3,aiJobTesterStream-1771867529384143969,172e98a3,2026-02-23 17:25:33.946,2026-02-23 17:26:18.933,13.060243,1
4,scenario_1_clean_success_no_swap_fps_gt_12,aiJobTesterStream-1771865445074714027|c86a1938,aiJobTesterStream-1771865445074714027,c86a1938,2026-02-23 16:50:50.349,2026-02-23 16:51:35.694,12.113868,1



scenario_2_no_orchestrator_then_closed: 5 candidate rows


,scenario_name,workflow_session_id,stream_id,request_id,session_start_ts,session_end_ts,startup_success,startup_excused,startup_unexcused,has_no_orch,has_close
0,scenario_2_no_orchestrator_then_closed,aiJobTesterStream-1771871232964100874|854a16ea,aiJobTesterStream-1771871232964100874,854a16ea,2026-02-23 18:27:17.551,2026-02-23 18:27:17.653,0,1,0,1,1
1,scenario_2_no_orchestrator_then_closed,aiJobTesterStream-1771871225859938419|d1ddc4b6,aiJobTesterStream-1771871225859938419,d1ddc4b6,2026-02-23 18:27:10.301,2026-02-23 18:27:10.808,0,1,0,1,1
2,scenario_2_no_orchestrator_then_closed,aiJobTesterStream-1771871218347697389|7aec8524,aiJobTesterStream-1771871218347697389,7aec8524,2026-02-23 18:27:02.835,2026-02-23 18:27:02.937,0,1,0,1,1
3,scenario_2_no_orchestrator_then_closed,aiJobTesterStream-1771871211641111413|d0237842,aiJobTesterStream-1771871211641111413,d0237842,2026-02-23 18:26:56.133,2026-02-23 18:26:58.808,0,1,0,1,1
4,scenario_2_no_orchestrator_then_closed,aiJobTesterStream-1771871204539578090|8e783882,aiJobTesterStream-1771871204539578090,8e783882,2026-02-23 18:26:49.035,2026-02-23 18:26:49.141,0,1,0,1,1



scenario_3_success_with_swap: 5 candidate rows


,scenario_name,workflow_session_id,stream_id,request_id,session_start_ts,session_end_ts,swap_count,segment_orchestrators
0,scenario_3_success_with_swap,aiJobTesterStream-1771867705548933117|57d02db0,aiJobTesterStream-1771867705548933117,57d02db0,2026-02-23 17:28:30.054,2026-02-23 17:29:38.497,1,1
1,scenario_3_success_with_swap,aiJobTesterStream-1771867626722162118|43d32181,aiJobTesterStream-1771867626722162118,43d32181,2026-02-23 17:27:11.312,2026-02-23 17:27:55.213,1,2
2,scenario_3_success_with_swap,aiJobTesterStream-1771862817891562471|44215189,aiJobTesterStream-1771862817891562471,44215189,2026-02-23 16:07:05.710,2026-02-23 16:08:06.729,1,1
3,scenario_3_success_with_swap,aiJobTesterStream-1771860563997073010|b402eec7,aiJobTesterStream-1771860563997073010,b402eec7,2026-02-23 15:29:28.529,2026-02-23 15:30:43.264,1,1
4,scenario_3_success_with_swap,aiJobTesterStream-1771849904589408897|01ffe674,aiJobTesterStream-1771849904589408897,01ffe674,2026-02-23 12:31:48.980,2026-02-23 12:33:03.425,1,1



scenario_4_success_with_param_updates: 0 candidate rows


""


In [12]:
scenario_assertions = run_assertion_file(SQL_DIR / 'assertions_scenario_candidates.sql')
display(scenario_assertions)

scenario_failures = scenario_assertions[scenario_assertions['status'] == 'FAIL']
print(f'Scenario assertion failures: {len(scenario_failures)}')


,test_name,status,failed_rows,candidates
0,scenario_1_clean_success_no_swap_fps_gt_12_exists,PASS,0,1
1,scenario_2_no_orchestrator_then_closed_exists,PASS,0,1
2,scenario_3_success_with_swap_exists,PASS,0,1
3,scenario_4_success_with_param_updates_exists,PASS,0,0


Scenario assertion failures: 0


## Interactive Session Edge Explorer

Select a scenario candidate session and render raw event/edge timeline dynamically.

In [13]:
# Expand and run this to build dropdown options from scenario candidate results
# Use a unique value (scenario_name, row_index, session_id) so duplicates are not collapsed.
session_options = []
for scenario_name, df in scenario_candidates.items():
    if df is None or df.empty or 'workflow_session_id' not in df.columns:
        continue
    for row_idx, row in df.reset_index(drop=True).iterrows():
        sid = str(row['workflow_session_id'])
        label = f"{scenario_name} | row={row_idx} | {sid[:28]}"
        session_options.append((label, (scenario_name, int(row_idx), sid)))

session_picker = widgets.Dropdown(
    options=session_options,
    description='Session:',
    layout=widgets.Layout(width='95%')
)
run_edges_btn = widgets.Button(description='Show Raw Edges', button_style='primary')
edge_out = widgets.Output()

def show_selected_session_edges(_):
    edge_out.clear_output()
    selected = session_picker.value
    if not selected:
        with edge_out:
            print('No scenario candidates in current window.')
        return

    scenario_name, row_idx, sid = selected

    sql = """
    WITH
      {sid:String} AS sid,
      latest_session AS
      (
        SELECT
          argMax(stream_id, version) AS stream_id,
          argMax(request_id, version) AS request_id
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      ),
      session_window AS
      (
        SELECT
          coalesce(argMax(session_start_ts, version), {from_ts:DateTime64(3)}) - INTERVAL 1 HOUR AS from_ts,
          coalesce(argMax(session_end_ts, version), argMax(session_start_ts, version), {to_ts:DateTime64(3)}) + INTERVAL 1 HOUR AS to_ts
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      )
    SELECT
      ts,
      raw_source,
      edge_name,
      stream_id,
      request_id,
      orchestrator_address,
      data_timestamp,
      raw_event_timestamp,
      toUInt8(edge_name = 'orchestrator_swap') AS is_swap_event,
      toUInt8(edge_name = 'params_update') AS is_params_update
    FROM
    (
      SELECT
        edge_ts AS ts,
        'fact_stream_trace_edges' AS raw_source,
        trace_type AS edge_name,
        stream_id,
        request_id,
        orchestrator_address,
        edge_ts AS data_timestamp,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS raw_event_timestamp
      FROM livepeer_analytics.fact_stream_trace_edges
      WHERE workflow_session_id = sid

      UNION ALL

      SELECT
        coalesce(data_timestamp, event_timestamp) AS ts,
        'raw_stream_trace_events' AS raw_source,
        trace_type AS edge_name,
        stream_id,
        request_id,
        orchestrator_address,
        data_timestamp,
        event_timestamp AS raw_event_timestamp
      FROM livepeer_analytics.stream_trace_events
      WHERE (request_id = (SELECT request_id FROM latest_session)
         OR stream_id = (SELECT stream_id FROM latest_session))
        AND event_timestamp >= (SELECT from_ts FROM session_window)
        AND event_timestamp < (SELECT to_ts FROM session_window)

      UNION ALL

      SELECT
        event_timestamp AS ts,
        'ai_stream_events' AS raw_source,
        event_type AS edge_name,
        stream_id,
        request_id,
        '' AS orchestrator_address,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS data_timestamp,
        event_timestamp AS raw_event_timestamp
      FROM livepeer_analytics.ai_stream_events
      WHERE (request_id = (SELECT request_id FROM latest_session)
         OR stream_id = (SELECT stream_id FROM latest_session))
        AND event_timestamp >= (SELECT from_ts FROM session_window)
        AND event_timestamp < (SELECT to_ts FROM session_window)

      UNION ALL

      SELECT
        event_timestamp AS ts,
        'ai_stream_status' AS raw_source,
        concat('state:', state) AS edge_name,
        stream_id,
        request_id,
        orchestrator_address,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS data_timestamp,
        event_timestamp AS raw_event_timestamp
      FROM livepeer_analytics.ai_stream_status
      WHERE (request_id = (SELECT request_id FROM latest_session)
         OR stream_id = (SELECT stream_id FROM latest_session))
        AND event_timestamp >= (SELECT from_ts FROM session_window)
        AND event_timestamp < (SELECT to_ts FROM session_window)
    ) t
    ORDER BY ts, raw_source, edge_name
    """

    try:
        df = query_df(sql, {**params, 'sid': sid})
        with edge_out:
            print(f'Scenario: {scenario_name} | row={row_idx} | session={sid}')
            if df.empty:
                print('No timeline rows found for selected session.')
            else:
                print(
                    f"swap_events_count={int(df['is_swap_event'].sum())} | "
                    f"params_update_events_count={int(df['is_params_update'].sum())}"
                )
            display(df)
    except Exception as exc:
        with edge_out:
            print(f'Query failed: {exc}')

run_edges_btn.on_click(show_selected_session_edges)

display(session_picker, run_edges_btn, edge_out)


Dropdown(description='Session:', layout=Layout(width='95%'), options=(('scenario_1_clean_success_no_swap_fps_g…

Button(button_style='primary', description='Show Raw Edges', style=ButtonStyle())

Output()

## Session Diagnostics (Swaps, Errors, Param Updates)

This section explains why a session can show `swap_count > 0` even when no explicit `orchestrator_swap` edge appears.

`swap_count` in session facts is derived as:
- explicit swaps from trace edge `orchestrator_swap`, OR
- fallback from observed orchestrator identity changes across session state/attribution updates.

In [14]:
def resolve_selected_session() -> tuple[str | None, str | None]:
    if 'session_picker' in globals():
        selected = getattr(session_picker, 'value', None)
        if selected:
            if isinstance(selected, tuple) and len(selected) == 3:
                scenario_name, row_idx, sid = selected
                return str(sid), f"interactive:{scenario_name}:row={row_idx}"
            return str(selected), 'interactive'

    for name, df in scenario_candidates.items():
        if not df.empty and 'workflow_session_id' in df.columns:
            return str(df.iloc[0]['workflow_session_id']), name

    fallback = query_df("""
    SELECT argMax(workflow_session_id, version) AS workflow_session_id
    FROM livepeer_analytics.fact_workflow_sessions
    WHERE session_start_ts >= {from_ts:DateTime64(3)}
      AND session_start_ts < {to_ts:DateTime64(3)}
    """, params)
    if fallback.empty or not fallback.iloc[0]['workflow_session_id']:
        return None, None
    return str(fallback.iloc[0]['workflow_session_id']), 'fallback_latest_session'

selected_session_id, selected_source = resolve_selected_session()
print('Selected session:', selected_session_id, 'from', selected_source)


Selected session: aiJobTesterStream-1771871101314442233|8381f156 from interactive:scenario_1_clean_success_no_swap_fps_gt_12:row=0


In [15]:
if selected_session_id:
    diagnostics_sql = """
    WITH
      {sid:String} AS sid,
      latest_session AS
      (
        SELECT
          argMax(version, version) AS latest_version,
          argMax(session_start_ts, version) AS session_start_ts,
          argMax(session_end_ts, version) AS session_end_ts,
          argMax(stream_id, version) AS stream_id,
          argMax(request_id, version) AS request_id,
          argMax(orchestrator_address, version) AS latest_orchestrator_address,
          argMax(known_stream, version) AS known_stream,
          argMax(startup_success, version) AS startup_success,
          argMax(startup_excused, version) AS startup_excused,
          argMax(startup_unexcused, version) AS startup_unexcused,
          argMax(swap_count, version) AS fact_swap_count,
          argMax(error_count, version) AS fact_error_count,
          argMax(excusable_error_count, version) AS fact_excusable_error_count
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      ),
      trace_counts AS
      (
        SELECT
          countIf(trace_type = 'orchestrator_swap') AS explicit_swap_edges,
          uniqExactIf(orchestrator_address, orchestrator_address != '') AS trace_orchestrators_seen
        FROM livepeer_analytics.fact_stream_trace_edges
        WHERE workflow_session_id = sid
      ),
      segment_counts AS
      (
        SELECT
          count() AS segment_rows,
          uniqExactIf(orchestrator_address, orchestrator_address != '') AS segment_orchestrators_seen
        FROM livepeer_analytics.fact_workflow_session_segments
        WHERE workflow_session_id = sid
      ),
      ai_counts AS
      (
        SELECT
          countIf(event_type = 'error') AS raw_error_events,
          countIf(event_type = 'params_update') AS raw_params_update_events
        FROM livepeer_analytics.ai_stream_events
        WHERE (request_id = (SELECT request_id FROM latest_session)
           OR stream_id = (SELECT stream_id FROM latest_session))
          AND event_timestamp >= {from_ts:DateTime64(3)}
          AND event_timestamp < {to_ts:DateTime64(3)}
      ),
      param_fact AS
      (
        SELECT count() AS fact_param_update_rows
        FROM livepeer_analytics.fact_workflow_param_updates
        WHERE workflow_session_id = sid
      ),
      session_versions AS
      (
        SELECT
          count() AS session_rows_all_versions,
          uniqExactIf(orchestrator_address, orchestrator_address != '') AS session_orchestrators_seen_across_versions
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      )
    SELECT
      latest_session.latest_version,
      latest_session.session_start_ts,
      latest_session.session_end_ts,
      latest_session.stream_id,
      latest_session.request_id,
      latest_session.latest_orchestrator_address,
      latest_session.known_stream,
      latest_session.startup_success,
      latest_session.startup_excused,
      latest_session.startup_unexcused,
      latest_session.fact_swap_count,
      trace_counts.explicit_swap_edges,
      trace_counts.trace_orchestrators_seen,
      segment_counts.segment_rows,
      segment_counts.segment_orchestrators_seen,
      ai_counts.raw_error_events,
      latest_session.fact_error_count,
      latest_session.fact_excusable_error_count,
      ai_counts.raw_params_update_events,
      param_fact.fact_param_update_rows,
      session_versions.session_rows_all_versions,
      session_versions.session_orchestrators_seen_across_versions
    FROM latest_session
    CROSS JOIN trace_counts
    CROSS JOIN segment_counts
    CROSS JOIN ai_counts
    CROSS JOIN param_fact
    CROSS JOIN session_versions
    """

    diagnostics_df = query_df(diagnostics_sql, {**params, 'sid': selected_session_id})
    display(diagnostics_df)
else:
    print('No session available in this window.')


,latest_version,session_start_ts,session_end_ts,stream_id,request_id,latest_orchestrator_address,known_stream,startup_success,startup_excused,startup_unexcused,...,trace_orchestrators_seen,segment_rows,segment_orchestrators_seen,raw_error_events,fact_error_count,fact_excusable_error_count,raw_params_update_events,fact_param_update_rows,session_rows_all_versions,session_orchestrators_seen_across_versions
0,12,2026-02-23 18:25:05.925,2026-02-23 18:25:49.266,aiJobTesterStream-1771871101314442233,8381f156,0xdc28f2842810d1a013ad51de174d02eaba192dc7,1,1,0,0,...,1,1,1,1,1,1,0,0,1,1


## Session GPU Correlation (Stream -> Session -> Capability)
For the selected session, this section shows:
- GPU IDs observed in session facts/segments/param updates
- capability correlation candidates using orchestrator + pipeline/model


In [16]:
if selected_session_id:
    gpu_observed_sql = """
    WITH
      {sid:String} AS sid,
      latest_session AS
      (
        SELECT
          argMax(stream_id, version) AS stream_id,
          argMax(request_id, version) AS request_id,
          argMax(orchestrator_address, version) AS orchestrator_address,
          argMax(pipeline, version) AS pipeline,
          argMax(model_id, version) AS model_id,
          argMax(gpu_id, version) AS gpu_id,
          argMax(session_start_ts, version) AS session_start_ts,
          argMax(session_end_ts, version) AS session_end_ts
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      ),
      segment_gpus AS
      (
        SELECT
          groupUniqArrayIf(ifNull(gpu_id, ''), ifNull(gpu_id, '') != '') AS segment_gpu_ids,
          groupUniqArrayIf(ifNull(model_id, ''), ifNull(model_id, '') != '') AS segment_model_ids
        FROM livepeer_analytics.fact_workflow_session_segments
        WHERE workflow_session_id = sid
      ),
      param_gpus AS
      (
        SELECT
          groupUniqArrayIf(ifNull(gpu_id, ''), ifNull(gpu_id, '') != '') AS param_update_gpu_ids,
          groupUniqArrayIf(ifNull(model_id, ''), ifNull(model_id, '') != '') AS param_update_model_ids
        FROM livepeer_analytics.fact_workflow_param_updates
        WHERE workflow_session_id = sid
      )
    SELECT
      sid AS workflow_session_id,
      latest_session.stream_id,
      latest_session.request_id,
      latest_session.orchestrator_address,
      latest_session.pipeline,
      latest_session.model_id,
      latest_session.gpu_id AS session_gpu_id,
      (SELECT segment_gpu_ids FROM segment_gpus) AS segment_gpu_ids,
      (SELECT param_update_gpu_ids FROM param_gpus) AS param_update_gpu_ids,
      (SELECT segment_model_ids FROM segment_gpus) AS segment_model_ids,
      (SELECT param_update_model_ids FROM param_gpus) AS param_update_model_ids,
      latest_session.session_start_ts,
      latest_session.session_end_ts
    FROM latest_session
    """

    capability_corr_sql = """
    WITH
      {sid:String} AS sid,
      latest_session AS
      (
        SELECT
          argMax(orchestrator_address, version) AS orchestrator_address,
          argMax(pipeline, version) AS pipeline,
          argMax(model_id, version) AS model_id,
          argMax(session_start_ts, version) AS session_start_ts,
          argMax(session_end_ts, version) AS session_end_ts
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      ),
      observed AS
      (
        SELECT
          groupUniqArrayIf(ifNull(gpu_id, ''), ifNull(gpu_id, '') != '') AS gpu_ids,
          groupUniqArrayIf(ifNull(model_id, ''), ifNull(model_id, '') != '') AS model_ids
        FROM livepeer_analytics.fact_workflow_session_segments
        WHERE workflow_session_id = sid
      )
    SELECT
      d.snapshot_ts,
      d.orchestrator_address,
      d.orchestrator_proxy_address,
      d.pipeline,
      d.model_id,
      d.gpu_id,
      d.gpu_name,
      d.runner_version,
      d.region,
      toUInt8(d.pipeline = (SELECT pipeline FROM latest_session)) AS pipeline_match,
      toUInt8(ifNull(d.model_id, '') = ifNull((SELECT model_id FROM latest_session), '') OR has((SELECT model_ids FROM observed), ifNull(d.model_id, ''))) AS model_match,
      toUInt8(has((SELECT gpu_ids FROM observed), ifNull(d.gpu_id, ''))) AS gpu_match
    FROM livepeer_analytics.dim_orchestrator_capability_snapshots d
    WHERE d.orchestrator_address = (SELECT orchestrator_address FROM latest_session)
      AND d.snapshot_ts >= (SELECT session_start_ts FROM latest_session) - INTERVAL 24 HOUR
      AND d.snapshot_ts <= coalesce((SELECT session_end_ts FROM latest_session), now64(3, 'UTC')) + INTERVAL 24 HOUR
      AND
      (
        d.pipeline = (SELECT pipeline FROM latest_session)
        OR ifNull(d.model_id, '') = ifNull((SELECT model_id FROM latest_session), '')
        OR has((SELECT model_ids FROM observed), ifNull(d.model_id, ''))
        OR has((SELECT gpu_ids FROM observed), ifNull(d.gpu_id, ''))
      )
    ORDER BY d.snapshot_ts DESC
    LIMIT 200
    """

    gpu_observed_df = query_df(gpu_observed_sql, {**params, 'sid': selected_session_id})
    display(gpu_observed_df)

    capability_corr_df = query_df(capability_corr_sql, {**params, 'sid': selected_session_id})
    print(f'Capability correlation rows: {len(capability_corr_df)}')
    display(capability_corr_df)
else:
    print('No session available in this window.')





,workflow_session_id,stream_id,request_id,orchestrator_address,pipeline,model_id,session_gpu_id,segment_gpu_ids,param_update_gpu_ids,segment_model_ids,param_update_model_ids,session_start_ts,session_end_ts
0,aiJobTesterStream-1771871101314442233|8381f156,aiJobTesterStream-1771871101314442233,8381f156,0xdc28f2842810d1a013ad51de174d02eaba192dc7,streamdiffusion-sdxl-v2v,streamdiffusion-sdxl,GPU-e7de8ed7-8ab1-2c85-db2f-633e8ccf6476,[GPU-e7de8ed7-8ab1-2c85-db2f-633e8ccf6476],[],[streamdiffusion-sdxl],[],2026-02-23 18:25:05.925,2026-02-23 18:25:49.266


Capability correlation rows: 200


,snapshot_ts,orchestrator_address,orchestrator_proxy_address,pipeline,model_id,gpu_id,gpu_name,runner_version,region,pipeline_match,model_match,gpu_match
0,2026-02-23 18:24:30.632,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-e7de8ed7-8ab1-2c85-db2f-633e8ccf6476,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,1
1,2026-02-23 18:18:39.871,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0
2,2026-02-23 18:17:49.296,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0
3,2026-02-23 18:17:32.560,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0
4,2026-02-23 17:59:30.632,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2026-02-22 21:59:30.631,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0
196,2026-02-22 21:53:39.870,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0
197,2026-02-22 21:52:49.295,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0
198,2026-02-22 21:52:32.560,0xdc28f2842810d1a013ad51de174d02eaba192dc7,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,live-video-to-video,streamdiffusion-sdxl,GPU-41d07375-ec9f-7995-6de0-fe8c6edb4e0b,NVIDIA GeForce RTX 4090,0.14.0,None,0,1,0


In [17]:
if selected_session_id:
    timeline_sql = """
    WITH
      {sid:String} AS sid,
      latest_session AS
      (
        SELECT
          argMax(stream_id, version) AS stream_id,
          argMax(request_id, version) AS request_id
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      ),
      session_window AS
      (
        SELECT
          coalesce(argMax(session_start_ts, version), {from_ts:DateTime64(3)}) - INTERVAL 1 HOUR AS from_ts,
          coalesce(argMax(session_end_ts, version), argMax(session_start_ts, version), {to_ts:DateTime64(3)}) + INTERVAL 1 HOUR AS to_ts
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      )
    SELECT
      ts,
      source,
      detail,
      stream_id,
      request_id,
      orchestrator_address,
      data_timestamp,
      raw_event_timestamp
    FROM
    (
      SELECT
        edge_ts AS ts,
        'fact_stream_trace_edges' AS source,
        trace_type AS detail,
        stream_id,
        request_id,
        orchestrator_address,
        edge_ts AS data_timestamp,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS raw_event_timestamp
      FROM livepeer_analytics.fact_stream_trace_edges
      WHERE workflow_session_id = sid

      UNION ALL

      SELECT
        coalesce(data_timestamp, event_timestamp) AS ts,
        'raw_stream_trace_events' AS source,
        trace_type AS detail,
        stream_id,
        request_id,
        orchestrator_address,
        data_timestamp,
        event_timestamp AS raw_event_timestamp
      FROM livepeer_analytics.stream_trace_events
      WHERE (request_id = (SELECT request_id FROM latest_session)
         OR stream_id = (SELECT stream_id FROM latest_session))
        AND event_timestamp >= (SELECT from_ts FROM session_window)
        AND event_timestamp < (SELECT to_ts FROM session_window)

      UNION ALL

      SELECT
        event_timestamp AS ts,
        'raw_ai_stream_events' AS source,
        event_type AS detail,
        stream_id,
        request_id,
        '' AS orchestrator_address,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS data_timestamp,
        event_timestamp AS raw_event_timestamp
      FROM livepeer_analytics.ai_stream_events
      WHERE (request_id = (SELECT request_id FROM latest_session)
         OR stream_id = (SELECT stream_id FROM latest_session))
        AND event_timestamp >= (SELECT from_ts FROM session_window)
        AND event_timestamp < (SELECT to_ts FROM session_window)
    ) t
    ORDER BY ts
    """

    timeline_df = query_df(timeline_sql, {**params, 'sid': selected_session_id})
    display(timeline_df.head(400))
else:
    print('No session available in this window.')


,ts,source,detail,stream_id,request_id,orchestrator_address,data_timestamp,raw_event_timestamp
0,2026-02-23 18:25:05.925,fact_stream_trace_edges,gateway_receive_stream_request,aiJobTesterStream-1771871101314442233,8381f156,,2026-02-23 18:25:05.925,NaT
1,2026-02-23 18:25:05.925,raw_stream_trace_events,gateway_receive_stream_request,aiJobTesterStream-1771871101314442233,8381f156,,2026-02-23 18:25:05.925,2026-02-23 18:25:05.926
2,2026-02-23 18:25:16.009,fact_stream_trace_edges,runner_send_first_processed_segment,aiJobTesterStream-1771871101314442233,8381f156,0xdc28f2842810d1a013ad51de174d02eaba192dc7,2026-02-23 18:25:16.009,NaT
3,2026-02-23 18:25:16.009,raw_stream_trace_events,runner_send_first_processed_segment,aiJobTesterStream-1771871101314442233,8381f156,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,2026-02-23 18:25:16.009,2026-02-23 18:25:16.057
4,2026-02-23 18:25:16.059,fact_stream_trace_edges,gateway_send_first_ingest_segment,aiJobTesterStream-1771871101314442233,8381f156,0xdc28f2842810d1a013ad51de174d02eaba192dc7,2026-02-23 18:25:16.059,NaT
5,2026-02-23 18:25:16.059,raw_stream_trace_events,gateway_send_first_ingest_segment,aiJobTesterStream-1771871101314442233,8381f156,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,2026-02-23 18:25:16.059,2026-02-23 18:25:16.059
6,2026-02-23 18:25:17.161,fact_stream_trace_edges,gateway_receive_first_processed_segment,aiJobTesterStream-1771871101314442233,8381f156,0xdc28f2842810d1a013ad51de174d02eaba192dc7,2026-02-23 18:25:17.161,NaT
7,2026-02-23 18:25:17.161,raw_stream_trace_events,gateway_receive_first_processed_segment,aiJobTesterStream-1771871101314442233,8381f156,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,2026-02-23 18:25:17.161,2026-02-23 18:25:17.161
8,2026-02-23 18:25:18.093,fact_stream_trace_edges,runner_receive_first_ingest_segment,aiJobTesterStream-1771871101314442233,8381f156,0xdc28f2842810d1a013ad51de174d02eaba192dc7,2026-02-23 18:25:18.093,NaT
9,2026-02-23 18:25:18.093,raw_stream_trace_events,runner_receive_first_ingest_segment,aiJobTesterStream-1771871101314442233,8381f156,0xd66e5c00725e0d87d57172191ebd5e865fd71cff,2026-02-23 18:25:18.093,2026-02-23 18:25:18.139


## Bronze -> Silver -> Gold Audit (Serving Validation)
For the selected session, this section traces counts and time bounds from bronze/raw events to silver facts and gold API views.
Use it to quickly verify that serving rows are grounded in source data.


In [18]:
if selected_session_id:
    session_ctx_sql = '''
    WITH {sid:String} AS sid
    SELECT
      argMax(stream_id, version) AS stream_id,
      argMax(request_id, version) AS request_id,
      argMax(orchestrator_address, version) AS orchestrator_address,
      argMax(pipeline, version) AS pipeline,
      argMax(model_id, version) AS model_id,
      argMax(gpu_id, version) AS gpu_id,
      argMax(session_start_ts, version) AS session_start_ts,
      argMax(session_end_ts, version) AS session_end_ts
    FROM livepeer_analytics.fact_workflow_sessions
    WHERE workflow_session_id = sid
    '''
    session_ctx = query_df(session_ctx_sql, {**params, 'sid': selected_session_id})
    display(session_ctx)

    trace_sql = '''
    WITH
      {sid:String} AS sid,
      ctx AS
      (
        SELECT
          argMax(stream_id, version) AS stream_id,
          argMax(request_id, version) AS request_id,
          argMax(orchestrator_address, version) AS orchestrator_address,
          argMax(pipeline, version) AS pipeline,
              argMax(model_id, version) AS model_id,
          argMax(gpu_id, version) AS gpu_id,
          argMax(session_start_ts, version) AS session_start_ts,
          argMax(session_end_ts, version) AS session_end_ts
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      ),
      w AS
      (
        SELECT
          toStartOfInterval(session_start_ts, INTERVAL 1 HOUR) - INTERVAL 1 HOUR AS from_h,
          toStartOfInterval(ifNull(session_end_ts, session_start_ts), INTERVAL 1 HOUR) + INTERVAL 2 HOUR AS to_h,
          stream_id,
          request_id,
          orchestrator_address,
          pipeline,
          model_id,
          gpu_id
        FROM ctx
      )
    SELECT *
    FROM
    (
      SELECT
        'bronze_ai_stream_status' AS stage,
        toUInt64(count()) AS rows,
        min(event_timestamp) AS min_ts,
        max(event_timestamp) AS max_ts,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS window_start,
        CAST(NULL AS Nullable(Float64)) AS avg_output_fps,
        CAST(NULL AS Nullable(UInt64)) AS status_samples,
        CAST(NULL AS Nullable(UInt64)) AS known_sessions
      FROM livepeer_analytics.ai_stream_status
      WHERE (stream_id = (SELECT stream_id FROM w) OR request_id = (SELECT request_id FROM w))
        AND event_timestamp >= (SELECT from_h FROM w)
        AND event_timestamp < (SELECT to_h FROM w)

      UNION ALL

      SELECT
        'bronze_stream_trace_events' AS stage,
        toUInt64(count()) AS rows,
        min(event_timestamp) AS min_ts,
        max(event_timestamp) AS max_ts,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS window_start,
        CAST(NULL AS Nullable(Float64)) AS avg_output_fps,
        CAST(NULL AS Nullable(UInt64)) AS status_samples,
        CAST(NULL AS Nullable(UInt64)) AS known_sessions
      FROM livepeer_analytics.stream_trace_events
      WHERE (stream_id = (SELECT stream_id FROM w) OR request_id = (SELECT request_id FROM w))
        AND event_timestamp >= (SELECT from_h FROM w)
        AND event_timestamp < (SELECT to_h FROM w)

      UNION ALL

      SELECT
        'silver_fact_stream_status_samples' AS stage,
        toUInt64(count()) AS rows,
        min(sample_ts) AS min_ts,
        max(sample_ts) AS max_ts,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS window_start,
        avg(toFloat64(output_fps)) AS avg_output_fps,
        toUInt64(count()) AS status_samples,
        CAST(NULL AS Nullable(UInt64)) AS known_sessions
      FROM livepeer_analytics.fact_stream_status_samples
      WHERE workflow_session_id = sid

      UNION ALL

      SELECT
        'silver_fact_workflow_latency_samples' AS stage,
        toUInt64(count()) AS rows,
        min(sample_ts) AS min_ts,
        max(sample_ts) AS max_ts,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS window_start,
        CAST(NULL AS Nullable(Float64)) AS avg_output_fps,
        CAST(NULL AS Nullable(UInt64)) AS status_samples,
        CAST(NULL AS Nullable(UInt64)) AS known_sessions
      FROM livepeer_analytics.fact_workflow_latency_samples
      WHERE workflow_session_id = sid

      UNION ALL

      SELECT
        'silver_fact_workflow_sessions_latest' AS stage,
        toUInt64(1) AS rows,
        argMax(session_start_ts, version) AS min_ts,
        argMax(session_end_ts, version) AS max_ts,
        CAST(NULL AS Nullable(DateTime64(3, 'UTC'))) AS window_start,
        CAST(NULL AS Nullable(Float64)) AS avg_output_fps,
        CAST(NULL AS Nullable(UInt64)) AS status_samples,
        toUInt64(argMax(known_stream, version)) AS known_sessions
      FROM livepeer_analytics.fact_workflow_sessions
      WHERE workflow_session_id = sid

      UNION ALL

      SELECT
        'gold_v_api_gpu_metrics_window' AS stage,
        toUInt64(count()) AS rows,
        min(window_start) AS min_ts,
        max(window_start) AS max_ts,
        min(window_start) AS window_start_min,
        avg(toFloat64(avg_output_fps)) AS avg_output_fps,
        sum(status_samples) AS status_samples,
        sum(known_sessions) AS known_sessions
      FROM livepeer_analytics.v_api_gpu_metrics
      WHERE window_start >= (SELECT from_h FROM w)
        AND window_start < (SELECT to_h FROM w)
        AND orchestrator_address = (SELECT orchestrator_address FROM w)
        AND pipeline = (SELECT pipeline FROM w)
        AND ifNull(model_id, '') = ifNull((SELECT model_id FROM w), '')
        AND ifNull(gpu_id, '') = ifNull((SELECT gpu_id FROM w), '')

      UNION ALL

      SELECT
        'gold_v_api_network_demand_window' AS stage,
        toUInt64(count()) AS rows,
        min(window_start) AS min_ts,
        max(window_start) AS max_ts,
        min(window_start) AS window_start_min,
        avg(toFloat64(avg_output_fps)) AS avg_output_fps,
        CAST(NULL AS Nullable(UInt64)) AS status_samples,
        sum(known_sessions) AS known_sessions
      FROM livepeer_analytics.v_api_network_demand
      WHERE window_start >= (SELECT from_h FROM w)
        AND window_start < (SELECT to_h FROM w)
        AND pipeline = (SELECT pipeline FROM w)
    )
    ORDER BY stage
    '''

    trace_df = query_df(trace_sql, {**params, 'sid': selected_session_id})
    display(trace_df)
else:
    print('No session available in this window.')


,stream_id,request_id,orchestrator_address,pipeline,model_id,gpu_id,session_start_ts,session_end_ts
0,aiJobTesterStream-1771871101314442233,8381f156,0xdc28f2842810d1a013ad51de174d02eaba192dc7,streamdiffusion-sdxl-v2v,streamdiffusion-sdxl,GPU-e7de8ed7-8ab1-2c85-db2f-633e8ccf6476,2026-02-23 18:25:05.925,2026-02-23 18:25:49.266


,stage,rows,min_ts,max_ts,window_start,avg_output_fps,status_samples,known_sessions
0,bronze_ai_stream_status,4,2026-02-23 18:25:16.226,2026-02-23 18:25:46.227,NaT,NaN,NaN,NaN
1,bronze_stream_trace_events,7,2026-02-23 18:25:05.926,2026-02-23 18:25:49.266,NaT,NaN,NaN,NaN
2,gold_v_api_gpu_metrics_window,1,2026-02-23 18:00:00.000,2026-02-23 18:00:00.000,2026-02-23 18:00:00,0.000000,0.0,2.0
3,gold_v_api_network_demand_window,4,2026-02-23 17:00:00.000,2026-02-23 18:00:00.000,2026-02-23 17:00:00,10.963575,NaN,19.0
4,silver_fact_stream_status_samples,4,2026-02-23 18:25:16.226,2026-02-23 18:25:46.227,NaT,13.758626,4.0,NaN
5,silver_fact_workflow_latency_samples,1,2026-02-23 18:25:05.925,2026-02-23 18:25:05.925,NaT,NaN,NaN,NaN
6,silver_fact_workflow_sessions_latest,1,2026-02-23 18:25:05.925,2026-02-23 18:25:49.266,NaT,NaN,NaN,1.0


## Gold Row Drill-Down (Hourly GPU View)
Pick a `v_api_gpu_metrics` hourly row for the selected session key and inspect the underlying silver inputs side-by-side.


In [19]:
if selected_session_id:
    gpu_hour_rows_sql = '''
    WITH
      {sid:String} AS sid,
      ctx AS
      (
        SELECT
          argMax(orchestrator_address, version) AS orchestrator_address,
          argMax(pipeline, version) AS pipeline,
          argMax(model_id, version) AS model_id,
          argMax(gpu_id, version) AS gpu_id,
          toStartOfInterval(argMax(session_start_ts, version), INTERVAL 1 HOUR) - INTERVAL 1 HOUR AS from_h,
          toStartOfInterval(ifNull(argMax(session_end_ts, version), argMax(session_start_ts, version)), INTERVAL 1 HOUR) + INTERVAL 2 HOUR AS to_h
        FROM livepeer_analytics.fact_workflow_sessions
        WHERE workflow_session_id = sid
      )
    SELECT
      window_start,
      orchestrator_address,
      pipeline,
      model_id,
      gpu_id,
      status_samples,
      avg_output_fps,
      prompt_to_first_frame_ms,
      startup_time_ms,
      e2e_latency_ms,
      known_sessions
    FROM livepeer_analytics.v_api_gpu_metrics
    WHERE window_start >= (SELECT from_h FROM ctx)
      AND window_start < (SELECT to_h FROM ctx)
      AND orchestrator_address = (SELECT orchestrator_address FROM ctx)
      AND pipeline = (SELECT pipeline FROM ctx)
      AND ifNull(model_id, '') = ifNull((SELECT model_id FROM ctx), '')
      AND ifNull(gpu_id, '') = ifNull((SELECT gpu_id FROM ctx), '')
    ORDER BY window_start
    '''

    gpu_hour_rows = query_df(gpu_hour_rows_sql, {**params, 'sid': selected_session_id})
    display(gpu_hour_rows)

    hour_options = [(str(r['window_start']), str(r['window_start'])) for _, r in gpu_hour_rows.iterrows()]
    hour_picker = widgets.Dropdown(options=hour_options, description='Hour:', layout=widgets.Layout(width='60%'))
    run_hour_btn = widgets.Button(description='Trace Hour Inputs', button_style='primary')
    hour_out = widgets.Output()

    def trace_hour_inputs(_):
        hour_out.clear_output()
        if not hour_picker.value:
            with hour_out:
                print('No hourly rows available for selected session key.')
            return

        sql = '''
        WITH
          {sid:String} AS sid,
          toDateTime64({hour:String}, 3, 'UTC') AS h,
          ctx AS
          (
            SELECT
              argMax(orchestrator_address, version) AS orchestrator_address,
              argMax(pipeline, version) AS pipeline,
                  argMax(model_id, version) AS model_id,
              argMax(gpu_id, version) AS gpu_id
            FROM livepeer_analytics.fact_workflow_sessions
            WHERE workflow_session_id = sid
          )
        SELECT *
        FROM
        (
          SELECT
            'silver_status_inputs' AS source,
            toUInt64(count()) AS rows,
            avg(toFloat64(output_fps)) AS avg_output_fps,
            CAST(NULL AS Nullable(Float64)) AS prompt_to_first_frame_ms,
            CAST(NULL AS Nullable(Float64)) AS startup_time_ms,
            CAST(NULL AS Nullable(Float64)) AS e2e_latency_ms,
            CAST(NULL AS Nullable(UInt64)) AS known_sessions
          FROM livepeer_analytics.fact_stream_status_samples
          WHERE sample_ts >= h AND sample_ts < h + INTERVAL 1 HOUR
            AND ifNull(orchestrator_address, '') = ifNull((SELECT orchestrator_address FROM ctx), '')
            AND ifNull(pipeline, '') = ifNull((SELECT pipeline FROM ctx), '')
            AND ifNull(model_id, '') = ifNull((SELECT model_id FROM ctx), '')
            AND ifNull(gpu_id, '') = ifNull((SELECT gpu_id FROM ctx), '')

          UNION ALL

          SELECT
            'silver_latency_inputs' AS source,
            toUInt64(count()) AS rows,
            CAST(NULL AS Nullable(Float64)) AS avg_output_fps,
            avgIf(prompt_to_first_frame_ms, has_prompt_to_first_frame = 1) AS prompt_to_first_frame_ms,
            avgIf(startup_time_ms, has_startup_time = 1) AS startup_time_ms,
            avgIf(e2e_latency_ms, has_e2e_latency = 1) AS e2e_latency_ms,
            CAST(NULL AS Nullable(UInt64)) AS known_sessions
          FROM livepeer_analytics.fact_workflow_latency_samples
          WHERE sample_ts >= h AND sample_ts < h + INTERVAL 1 HOUR
            AND ifNull(orchestrator_address, '') = ifNull((SELECT orchestrator_address FROM ctx), '')
            AND ifNull(pipeline, '') = ifNull((SELECT pipeline FROM ctx), '')
            AND ifNull(model_id, '') = ifNull((SELECT model_id FROM ctx), '')
            AND ifNull(gpu_id, '') = ifNull((SELECT gpu_id FROM ctx), '')

          UNION ALL

          SELECT
            'silver_session_reliability_inputs' AS source,
            toUInt64(count()) AS rows,
            CAST(NULL AS Nullable(Float64)) AS avg_output_fps,
            CAST(NULL AS Nullable(Float64)) AS prompt_to_first_frame_ms,
            CAST(NULL AS Nullable(Float64)) AS startup_time_ms,
            CAST(NULL AS Nullable(Float64)) AS e2e_latency_ms,
            sum(toUInt64(known_stream)) AS known_sessions
          FROM
          (
            SELECT
              workflow_session_id,
              argMax(known_stream, version) AS known_stream
            FROM livepeer_analytics.fact_workflow_sessions
            WHERE toStartOfInterval(session_start_ts, INTERVAL 1 HOUR) = h
              AND ifNull(orchestrator_address, '') = ifNull((SELECT orchestrator_address FROM ctx), '')
              AND ifNull(pipeline, '') = ifNull((SELECT pipeline FROM ctx), '')
                AND ifNull(model_id, '') = ifNull((SELECT model_id FROM ctx), '')
              AND ifNull(gpu_id, '') = ifNull((SELECT gpu_id FROM ctx), '')
            GROUP BY workflow_session_id
          ) x

          UNION ALL

          SELECT
            'gold_v_api_gpu_metrics' AS source,
            toUInt64(1) AS rows,
            avg_output_fps,
            prompt_to_first_frame_ms,
            startup_time_ms,
            e2e_latency_ms,
            known_sessions
          FROM livepeer_analytics.v_api_gpu_metrics
          WHERE window_start = h
            AND ifNull(orchestrator_address, '') = ifNull((SELECT orchestrator_address FROM ctx), '')
            AND ifNull(pipeline, '') = ifNull((SELECT pipeline FROM ctx), '')
            AND ifNull(model_id, '') = ifNull((SELECT model_id FROM ctx), '')
            AND ifNull(gpu_id, '') = ifNull((SELECT gpu_id FROM ctx), '')
        )
        ORDER BY source
        '''

        with hour_out:
            display(query_df(sql, {**params, 'sid': selected_session_id, 'hour': hour_picker.value}))

    run_hour_btn.on_click(trace_hour_inputs)
    display(widgets.VBox([hour_picker, run_hour_btn, hour_out]))
else:
    print('No session available in this window.')


,window_start,orchestrator_address,pipeline,model_id,gpu_id,status_samples,avg_output_fps,prompt_to_first_frame_ms,startup_time_ms,e2e_latency_ms,known_sessions
0,2026-02-23 18:00:00,0xdc28f2842810d1a013ad51de174d02eaba192dc7,streamdiffusion-sdxl-v2v,streamdiffusion-sdxl,GPU-e7de8ed7-8ab1-2c85-db2f-633e8ccf6476,0,0.0,11133.0,2452.0,13585.0,2


## CLI Harness Commands

Run the same checks outside notebook:

```bash
python scripts/run_clickhouse_query_pack.py --lookback-hours 24
python scripts/run_clickhouse_data_tests.py --sql-file tests/integration/sql/assertions_pipeline.sql --lookback-hours 24
python scripts/run_clickhouse_data_tests.py --sql-file tests/integration/sql/assertions_scenario_candidates.sql --lookback-hours 720
```

Export production fixtures for your four scenarios:

```bash
python scripts/export_scenario_fixtures.py \
  --from-ts 2026-01-01T00:00:00Z \
  --to-ts 2026-02-16T00:00:00Z \
  --limit-per-scenario 3
```